In [1]:
# Filename: 2.0-LeNet-5-MNIST.ipynb
# Author: Eyosyas Dagnachew
# Description: Train LeNet-5 model on MNIST dataset. Remember, the purpose of this implementation is not to create the most optimal network to classift the MNIST dataset. 
#              Instead, it is to learn how to read a machine learning research paper and implement it (and in the process have a better understanding of the paper and its methodology).

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
# Parameters/Hyperparameters
NUM_CLASSES = 10
BATCH_SIZE = 100        # 5000 (in the paper)
NUM_EPOCHS = 20          # 2 (in the paper)
LEARNING_RATE = 0.0005  # changing lr with iterations (in the paper)
MOMENTUM = 0.02         # 0.02 (in the paper)

In [5]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root="../../data", 
                                           train=True, 
                                           transform=transforms.Compose([transforms.ToTensor()]),
                                           download=False)
test_dataset = torchvision.datasets.MNIST(root="../../data",
                                          train=False,
                                          transform=transforms.Compose([transforms.ToTensor()]),
                                          download=False)
train_dataset, test_dataset

(Dataset MNIST
     Number of datapoints: 60000
     Root location: ../../data
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
            ),
 Dataset MNIST
     Number of datapoints: 10000
     Root location: ../../data
     Split: Test
     StandardTransform
 Transform: Compose(
                ToTensor()
            ))

In [6]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=False)

In [7]:
# LeNet-5 model
class LeNet5(nn.Module):
    '''
    Input: 32x32 pixel image in the paper, but 28x28 in the dataset
           The paper mentions that "[32x32] is significantly larger than the largest character in the
           database (at most 20x20 pixels centered in a 28x28 field). This might explain why the 
           the images in this dataset have been cropped to 28x28. Using 2D convolution because the 
           input is technically a 3D (32x32x1) image.
           
           Activation Function: In the paper, the activation function used is the scaled hyperbolic
                                tangent: f(a) = A*tanh(S*a), where A=1.71519 is the amplitude of the
                                function and S determines its slope at the origin. This is a sigmoidal
                                function. In my implmenetaion, I will initially just use the built-in
                                nn.Sigmoid() actiation function, and evntually, I will implement the 
                                function used in the paper.
           
    Output: 10
    '''
    
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        
        # Layer C1: conv layer with 6 28x28 feature maps with 5x5 kernels 
        #           parameters and connections: 156 trainable parameters, 122304 connections (28*28*156)
        #           notes: padding=2 because the original images were 32x32 but this dataset contains 28x28 images (2 pixels removed 
        #                  from all sides), so we have to make up for the removed pixels but adding a padding of 2 on all sides
        #           in: 32x32x1, out: 28x28x6
        self.c1 = nn.Sequential(
                    nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),
                    nn.Sigmoid()
        )
        
        # Layer S2: sub-sampling (pooling) layer with 6 14x14 feature maps with 2x2 kernels
        #           parameters and connections: 12 trainable parameters, 5800 connections (in the paper);
        #                                       0 trainable parameters, 5880 connections (in my implementation, read notes below)
        #           notes: "The four inputs to a unit in S2 are added, then multiplied by a trainable coefficient, 
        #                  and added to a trainable bias." This is where the difference between subsampling and pooling comes to play. 
        #                  Subsampling, as mentioned in the paper, is simply average pooling with learnable weights per feature map. 
        #                  In the Lua implementation of Torch, there is nn.SpatialSubSampling() but there is no such implementation
        #                  for PyTorch, so I will just use average pooling, i.e. AvgPool2d().
        #           in: 28x28x6, out: 14x14x6
        self.s2 = nn.Sequential(
                    nn.AvgPool2d(kernel_size=2, stride=2),    # sub-sampling
                    nn.Sigmoid()                              # sigmoidal function
        )
        
        # Layer C3: conv layer with 16 10x10 feature maps with 5x5 kernels
        #           parameters and connections: 1,516 trainable parameters and 151,600 connections (in the paper);
        #                                       2,416 trainable parameters and 241,600 connections (in my implementation, read notes below)
        #           notes: "Each unit in each feature map is connected to several 5x5 neighborhoods at identical locations in a subset
        #                  of S2's feature maps. Table I shows the set of S2 feature maps combined by each C3 feature maps." Instead of
        #                  connecting each output channel with each input channel, they connect certain output channels with certain input 
        #                  channels (detailed in Table 1). This is done because of two reasons: (1) To reduce the number of connections, 
        #                  and (2) To force a break of symmetry in the network so that "different feature maps are forced to extract 
        #                  different (hopefully complementary) features because they get different sets of input." I have no idea how to do
        #                  this in PyTorch (or if there even is a way), so I'm just going to use the default (a.k.a. groups=1).
        #           in: 14x14x6, out: 10x10x16
        self.c3 = nn.Sequential(
                    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, groups=1),
                    nn.Sigmoid()
        )
        # Layer S4: sub-sampling (pooling) layer with 16 5x5 feature maps with 2x2 kernels
        #           parameters and connections: 32 trainable parameters, 2,000 connections (in the paper)
        #                                       0 trainable parameters, 2,000 connections (in my implementation, read Layer S2 notes)
        #           notes: Connections between input and output are made in a similar way as C1 and S2.
        #           in: 10x10x16, out: 5x5x16
        self.s4 = nn.Sequential(
                    nn.AvgPool2d(kernel_size=2, stride=2),
                    nn.Sigmoid()
        )
        
        # Layer C5: conv layer with 120 1x1 feature maps with 5x5 kernels
        #           parameters and connections: 48,120 trainable connections (in the paper)
        #                                       48,120 trainable connections (in my implementation)
        #           notes: Since the size of S4 is the same as the size of the kernels of C5 (5x5), the size of C5's feature maps is 1x1. 
        #                  This makes the connection between S4 and C5 a full connection. However, C5 is labelled as a conv layer instead
        #                  of a fully-connected layer because "if LeNet-5 were made bigger and everything else kept constant, the feature 
        #                  map dimensions would be larger than 1x1."
        #           in: 5x5x16, out: 1x1x120
        self.c5 = nn.Sequential(
                    nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=0),
                    nn.Sigmoid()
        )
        
        # Layer F6: fully-connected layer with 84 units
        #           parameters and connections: 10,164 trainable parameters (in the paper)
        #                                       10,164 trainable parameters (in my implementation)
        #           notes: why 84 units? explained in paper
        #           in: 1x1x120, out: 1x1x84
        self.f6 = nn.Sequential(
                    nn.Linear(in_features=120, out_features=84),
                    nn.Sigmoid()
        )
        
        # Output Layer: Euclidean Radial Basis Function units (RBF), one for each class, with 84 inputs each
        #               notes: "Each output RBF unit computer the Euclidean distance between its input vector and its parameter vector. 
        #                      The further away is the input from the parameter vector, the larget is the RBF output. The output of a 
        #                      particular RBF can be interpreted as a penalty term measuring the fit between the input pattern and a model
        #                      of the class associated with the RBF... Given an input pattern, the loss function should be designed so as 
        #                      to get the configuration of F6 as close as possible to the parameter vector of the RBF that corresponds to
        #                      the pattern's desired class. The parameter vectors of these units were chose by hand and kept fixed..."
        #                      In the future, I will come back to this and try to have a better understanding of this, but for now I will 
        #                      just use another fully-connected layer with the built-in nn.Linear() function and a Softmax activation
        #                      layer for the probabilities. dim=-1 picks the last dimension (NUM_CLASSES) as the one to compute Softmax on.
        #               in: 1x1x84, out: 1x1xNUM_CLASSES
        self.output = nn.Sequential(
                        nn.Linear(in_features=84, out_features=NUM_CLASSES),
                        nn.Softmax(dim=-1)
        )
    
    def forward(self, x):   # x.shape = torch.Size([100, 1, 28, 28])  (100 == batch_size)
        out = self.c1(x)    # torch.Size([100, 6, 28, 28])
        out = self.s2(out)  # torch.Size([100, 6, 14, 14])
        out = self.c3(out)  # torch.Size([100, 16, 10, 10])
        out = self.s4(out)  # torch.Size([100, 16, 5, 5])
        out = self.c5(out)  # torch.Size([100, 120, 1, 1])
        out = out.reshape(out.size(0), -1)  # torch.Size([100, 120])
        out = self.f6(out)  # torch.Size([100, 84])
        out = self.output(out)  # torch.Size([100, 10])
        return out

In [8]:
# Initialize LeNet model
model = LeNet5(num_classes=NUM_CLASSES).to(device)
print(model)

# Print the number of parameters
# for parameter in model.parameters():
#     print(parameter.numel())    # 6 filters (150 each because 150/6 = 25 and 5x5 = 25 and weight sharing)

LeNet5(
  (c1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
  )
  (s2): Sequential(
    (0): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (1): Sigmoid()
  )
  (c3): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
  )
  (s4): Sequential(
    (0): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (1): Sigmoid()
  )
  (c5): Sequential(
    (0): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
  )
  (f6): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): Sigmoid()
  )
  (output): Sequential(
    (0): Linear(in_features=84, out_features=10, bias=True)
    (1): Softmax(dim=-1)
  )
)


In [9]:
# Initialize loss function and optimizer
criterion = nn.MSELoss()    # requires one-hot encoding
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

In [10]:
# One hot encoding buffer that I will reuse to store one-hot encoding of the labels
labels_onehot = torch.FloatTensor(BATCH_SIZE, NUM_CLASSES)

In [11]:
# Train the model
NUM_ITERATIONS = len(train_loader)
for epoch in range(NUM_EPOCHS):
    for iteration, training_batch in enumerate(train_loader):
        # Split training batch data into images and labels
        images, labels = training_batch
        images = images.to(device)
        labels = labels.to(device)
        
        # Convert labels to one-hot encoding to pass to the MSE loss function
        labels_onehot.zero_()
        labels_onehot.scatter_(1, labels.reshape(-1,1), 1)
        
        # Forward pass and calculate loss
        outputs = model(images)
        loss = criterion(outputs, labels_onehot)
        
        # Zero the gradient buffers
        optimizer.zero_grad()
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        if (iteration+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, NUM_EPOCHS, iteration+1, NUM_ITERATIONS, loss.item()))

Epoch [1/20], Step [100/600], Loss: 0.0905
Epoch [1/20], Step [200/600], Loss: 0.0907
Epoch [1/20], Step [300/600], Loss: 0.0902
Epoch [1/20], Step [400/600], Loss: 0.0910
Epoch [1/20], Step [500/600], Loss: 0.0902
Epoch [1/20], Step [600/600], Loss: 0.0907
Epoch [2/20], Step [100/600], Loss: 0.0913
Epoch [2/20], Step [200/600], Loss: 0.0921
Epoch [2/20], Step [300/600], Loss: 0.0910
Epoch [2/20], Step [400/600], Loss: 0.0905
Epoch [2/20], Step [500/600], Loss: 0.0906
Epoch [2/20], Step [600/600], Loss: 0.0897
Epoch [3/20], Step [100/600], Loss: 0.0907
Epoch [3/20], Step [200/600], Loss: 0.0908
Epoch [3/20], Step [300/600], Loss: 0.0902
Epoch [3/20], Step [400/600], Loss: 0.0903
Epoch [3/20], Step [500/600], Loss: 0.0911
Epoch [3/20], Step [600/600], Loss: 0.0907
Epoch [4/20], Step [100/600], Loss: 0.0907
Epoch [4/20], Step [200/600], Loss: 0.0902
Epoch [4/20], Step [300/600], Loss: 0.0904
Epoch [4/20], Step [400/600], Loss: 0.0907
Epoch [4/20], Step [500/600], Loss: 0.0904
Epoch [4/20

KeyboardInterrupt: 

In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

In [ ]:
# Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')